# Logistic Regression: Multiclass classification
M2U5 - Exercise 6

## What are we going to do?
- We will create a synthetic dataset for multiclass logistic regression
- We will preprocess the data
- We will train the model on the training subset and check its suitability
- We will find the optimal lambda regularization parameter using CV
- We will make predictions about new examples

Remember to follow the instructions for the submission of assignments indicated in [Submission Instructions](https://github.com/Tokio-School/Machine-Learning-EN/blob/main/Submission_instructions.md).

## Instructions
Having implemented the full training of a regularised logistic regression model for binary (2 classes) classification, we will repeat the same example for multiclass classification (3+ classes).

In [ ]:
import time
import numpy as np

from matplotlib import pyplot as plt

## Create a synthetic dataset for multiclass logistic regression

We will create a synthetic 3-class dataset for this complete implementation.

To do this, manually create a synthetic dataset for logistic regression with bias and error term (to have *Theta_true* available) with a slightly different code template than the one you used in the last exercise.

For the multiclass classification we will calculate Y in a different way: And it will have 2D (m x classes) dimensions, to represent all possible classes. We call this encoding of e.g. [0, 0, 1] for the 3/3 class "one-hot encoding":

- For each example and class, calculate *Y* with the sigmoid with *Theta_true* and *X*.
- Transform the values of *Y* to be `0` o `1` according to the max. value of the sigmoid of all the classes.
- Finally, transform the value of the class to 1 with a maximum value of the sigmoid, and the values of the other classes to 0, with a final ndarray for each example.

To introduce an error term, it runs through all *Y* values and changes the class of that example to a random class with a random error rate.

*NOTE:* Investigate how a synthetic dataset for multiclass classification could be achieved using Scikit-learn methods.

### Implement the sigmoid activation function

Copy your function from previous exercises:

In [ ]:
# TODO: Implement the sigmoid function

Create the synthetic dataset:

In [ ]:
# TODO: Manually generate a synthetic dataset with a bias term and an error term
# Since we are going to train so many models, generate a "small" dataset in order to train them quickly
# If you need to, you can make it even smaller, or if you want more accuracy and a more realistic challenge, make it bigger
m = 1000
n = 2
classes = 3

# Generate a 2D m x n array with random values between -1 and 1
# Insert a bias term as a first column of 1s
X = [...]

# Generate a 2D theta array with (classes x n + 1) random values
Theta_true = [...]

# Y shall have 2D dimensions of (m x classes)
# Calculate Y with the sigmoid and transform its values to 0 or 1 and then to one-hot encoding
for i in range(m):
    for c in range(classes):
        sigmoid_example = sigmoid([...])
        # Assign the only class corresponding to the example according to the max. value of the sigmoid
        Y[...] = [...]

# To introduce an error term, go through all the Y values and change
# the class chosen from that example to another random class with a random % error
# Note: make sure that the other random class representing the error is different from the original one
error = 0.15

for j in range(m):
    # If a random number is less than or equal to the error
    if [...]:
        # Assign a randomly selected class
        Y[...] = [...]

# Check the values and dimensions of the vectors
print('Theta to be estimated:')
print()

print('First 10 rows and 5 columns of X and Y:')
print()
print()

print('Dimensions of X and Y:')
print()

## Preprocess the data

As we did for linear regression, we will preprocess the data completely, following the usual 3 steps:

- Randomly reorder the data..
- Normalise the data..
- Divide the dataset into training, validation, and test subsets.

### Randomly reorder the dataset

Reorder the data in the *X* and *Y* dataset:

In [ ]:
# TODO: Reordena aleatoriamente el dataset

print('Primeras 10 filas y 5 columnas de X e Y:')
print()
print()

print('Reordenamos X e Y:')
# Usa un estado aleatorio inicial de 42, para mantener la reproducibilidad
X, Y = [...]

print('Primeras 10 filas y 5 columnas de X e Y:')
print()
print()

print('Dimensiones de X e Y:')
print()

### Normalizar el dataset

Implementa la función de normalización y normaliza el dataset de ejemplos *X*:

In [ ]:
# TODO: Normaliza el dataset con una función de normalización

# Copia tu función de normalización utilizada en la unidad de regresión lineal
def normalize(x, mu, std):
    pass

# Halla la media y la desviación típica de las características de X (columnas), excepto la primera (bias)
mu = [...]
std = [...]

print('X original:')
print(X)
print(X.shape)

print('Media y desviación típica de las características:')
print(mu)
print(mu.shape)
print(std)
print(std.shape)

print('X normalizada:')
X_norm = np.copy(X)
X_norm[...] = normalize(X[...], mu, std)    # Normaliza sólo la columna 1 y siguientes, no la 0
print(X_norm)
print(X_norm.shape)

*Nota*: Si habías modificado tu función *normalize* para que calculara y devolviera los valores de *mu* y *std*, puedes modificar esta celda para incluir tu código personalizado.

### Dividir el dataset en subsets de entrenamiento, validación y test

Divide el dataset de *X* e *Y* en 3 subsets con el ratio habitual, 60%/20%/20%.

Si tu nº de ejemplos es mucho más alto o bajo, siempre puedes modificar este ratio por otro como 50/25/25 o 80/10/10.

In [ ]:
# TODO: Divide el dataset X e Y en los 3 subsets según los ratios indicados

ratio = [60, 20, 20]
print('Ratio:\n', ratio, ratio[0] + ratio[1] + ratio[2])

r = [0, 0]
# Consejo: la función round() y el atributo x.shape pueden serte útiles
r[0] = [...]
r[1] = [...]
print('Índices de corte:\n', r)

# Consejo: la función np.array_split() puede serte útil
X_train, X_val, X_test = [...]
Y_train, Y_val, Y_test = [...]

print('Tamaños de los subsets:')
print(X_train.shape)
print(Y_train.shape)
print(X_val.shape)
print(Y_val.shape)
print(X_test.shape)
print(Y_test.shape)

## Entrenar un modelo inicial para cada clase

Para la clasificación multiclase, debemos entrenar un modelo diferente para cada clase. Por tanto, si tenemos 3 clases debemos entrenar 3 modelos diferentes.

Cada modelo sólo considerará los valores de la variable objetivo relativos a su clase de una forma binaria, clasificando los ejemplos como pertenecientes a su clase o no.

Para ello, sólo le proporcionaremos los valores de *Y* para dicha clase o columna. P. ej., para `Y = [[1, 0, 1], [0, 1, 0], [0, 0, 1]]`:
- *Y* para el modelo 1: `[1, 0, 0]`
- *Y* para el modelo 2: `[0, 1, 0]`
- *Y* para el modelo 3: `[0, 0, 1]`

Al igual que hacíamos en ejercicios anteriores, vamos a entrenar modelos iniciales para comprobar que nuestra implementación es correcta:
- Entrena un modelo inicial sin regularización para cada clase.
- Representa el histórico de la función de coste para comprobar su evolución para cada modelo.
- Si es necesario, modifica cualquier hiper-parámetro, como el ratio de entrenamiento, y reentrena los modelos. Usarás dichos hiper-parámetros en siguientes puntos.

Copia las celdas de ejercicios anteriores donde implementabas la función de coste y gradient descent regularizados para regresión logística y la celda donde entrenabas el modelo:

In [ ]:
# TODO: Copia las celdas con las funciones de coste y gradient descent para clasificación regularizadas

In [ ]:
# TODO: Entrena tus modelos sobre el subset de entrenamiento sin regularizar

# Crea una theta inicial con un valor dado, que puede ser el mismo para todos los modelos o no
theta_ini = [...]

print('Theta inicial:')
print(theta_ini)

alpha = 1e-1
lambda_ = 0.
e = 1e-3
iter_ = 1e3

print('Hiper-arámetros usados:')
print('Alpha:', alpha, 'Error máx.:', e, 'Nº iter', iter_)

# Inicializa unas variables para almacenar el resultado de cada modelo con las dimensiones adecuadas
# Cuidado: los modelos pueden necesitar un nº de iteraciones hasta que convergen diferente
# Dale a j_train un tamaño para almacenar hasta el nº máx. de iteraciones, aunque no se rellenen todos los elementos
j_train_ini = [...]
theta_train = [...]

t = time.time()
for c in [...]:    # Itera sobre el nº de clases
    print('\nModelo para la clase nº:', c)
    
    theta_train = [...]    # Copia profunda de theta_ini para que no se modifique
    
    t_model = time.time()
    j_train_ini[...], theta_train[...] = regularized_logistic_gradient_descent([...])
    
    print('Tiempo de entrenamiento para el modelo (s):', time.time() - t_model)
    
print('Tiempo de entrenamiento total (s):', time.time() - t)

print('\nCoste final del modelo para cada clase:')
print()

print('\nTheta final del modelo para cada clase:')
print()

In [ ]:
# TODO: Representa la evolución de la función de coste vs el nº de iteraciones para cada modelo

plt.figure(1)

plt.title('Función de coste en cada clase')

for c in range(clases):
    plt.subplot(clases, 1, c + 1)
    plt.xlabel('Iteraciones')
    plt.ylabel('Coste clase {}'.format(c))
    plt.plot(j_train_ini[...])

plt.show()

### Comprobar la idoneidad de los modelos

Revisa la precisión de tus modelos y modifica los parámetros para reentrenarlos si es necesario.

Recuerda que si tu dataset es "demasiado preciso" puedes volver a la celda original e introducir un término de error superior.

Por complejidad de una clasificación multiclase, no te pediremos en esta ocasión que compruebes si los modelos pueden estar sufriendo desviación o sobreajuste.

## Hallar el hiper-parámetro *lambda* óptimo por validación

Del mismo modo que hemos hecho en ejercicios anteriores, vamos a optimizar nuestro parámetro de regularización por validación para cada una de las clases y modelos.

Para ello, para cada clase, vamos a entrenar un modelo diferente por cada valor de *lambda* a considerar sobre el subset de entrenamiento, y evaluar su error o coste final sobre el subset de validación.

De nuevo, vamos a representar gráficamente el error de cada modelo vs el valor de *lambda* usado e implementar un código que elija automáticamente el modelo más óptimo de entre todos para cada clase.

Recuerda entrenar todos tus modelos en igualdad de condiciones, con los mismos hiper-parámetros.

Por tanto, ahora debes modificar el código de la celda anterior para no entrenar un modelo como antes sino uno por clase y por cada uno de los valores de *lambda* a considerar:

In [ ]:
# TODO: Entrena un modelo por cada valor de lambda diferente sobre X_train y evalúalo sobre X_val

# Usa de nuevo un espacio logarítmico entre 10 y 10^3 de 5 elementos con valores que comiencen por un decimal no-cero 1 o 3
# Al entrenar más modelos, podemos evaluar menos valores de lambda para reducir el tiempo de entrenamiento
lambdas = [...]

# Completa el código para entrenar un modelo diferente para cada clase y valor de lambda sobre X_train
# Almacena sus thetas y costes finales
# Posteriormente, evalúa sus costes totales en el subset de validación

# Almacena dicha información en los siguientes arrays
# Cuidado con sus dimensiones necesarias
j_train = [...]
j_val = [...]
theta_val = [...]

In [ ]:
# TODO: Representa gráficamente el error final para cada valor de lambda con una gráfica por clase

plt.figure()

# Completa con tu código
for c in range(clases):
    plt.subplot(clases, 1, c + 1)
    
    plt.title('Clase:', c)
    plt.xlabel('Lambda')
    plt.ylabel('Coste final')
    plt.plot(j_train[...])
    plt.plot(j_val[...])

plt.show()

### Escoger el mejor modelo para cada clase

Copia el código de ejercicios anteriores y modifícalo para escoger el modelo con mayor precisión sobre el subset de validación para cada clase:

In [ ]:
# TODO: Escoge los modelos y valores de lambda óptimos, con el menor error sobre el subset de validación

# Itera sobre todas las combinaciones de theta y lambda y escoge los modelos de menor coste en el subset de validación para cada clase

j_final = [...]
theta_final = [...]
lambda_final = [...]

## Evaluar los modelos sobre el subset de test

Finalmente, vamos a evaluar el modelo de cada clase sobre un subset de datos que no hemos usado para entrenarlos ni para escoger ningún hiper-parámetro.

Para ello, vamos a calcular el coste o error total sobre el subset de test y comprobar gráficamente los residuos sobre el mismo.

Recuerda usar sólo las columnas de la *Y* que "vería" cada modelo, puesto que clasifica los ejemplos en función de si pertenecen a su clase o no.

In [ ]:
# TODO: Calcula el error de los modelos sobre el subset de test usando la función de coste
# Utiliza la theta y lambda del modelo específico de la clase correspondiente a dicho ejemplo

j_test = [...]

In [ ]:
# TODO: Calcula las predicciones de los modelos sobre el subset de test, calcula los residuos y represéntalos

# Recuerda usar la función sigmoide para transformar las predicciones y escoger la clase según el valor máx. del sigmoide
Y_test_pred = [...]

residuos = [...]

plt.figure(4)

# Completa con tu código

plt.show()

## Realizar predicciones sobre nuevos ejemplos

Con nuestros modelos ya entrenados, optimizados y evaluados, lo único que nos queda es utilizarlos realizando predicciones con nuevos ejemplos.

Para ello, vamos a:
- Generar un nuevo ejemplo, siguiendo el mismo patrón que el dataset original.
- Normalizar sus características antes de poder realizar predicciones sobre ellos.
- Generar una predicción para dicho nuevo ejemplo para cada una de las clases, para cada uno de los 3 modelos.
- Escoger la clase final como la clase con mayor valor de *Y* tras el sigmoide, aunque varios modelos predijeran `Y >= 0.0; Y = 1`.

In [ ]:
# TODO: Genera un nuevo ejemplo siguiendo el patrón original, con término de bias

X_pred = [...]

# Para comparar, antes de normalizar los datos, usa la Theta_verd para ver cuál sería la clase real asociada
Y_verd = [...]

# Normaliza sus características (excepto el término de bias) con las medias y desviaciones típicas del subset de entrenamiento
X_pred = [...]

# Genera una predicción para dicho ejemplo para cada modelo usando el sigmoide
Y_pred = [...]

# Escoge la clase final como la de mayor valor tras el sigmoide y transfórmala a un vector one-hot encoding de 0 y 1
Y_pred = [...]

# Compara la clase real asociada a dicho nuevo ejemplo y la clase predicha
print('Clase real del nuevo ejemplo y clase predicha:')
print(Y_verd)
print(Y_pred)